<a href="https://colab.research.google.com/github/anirban1221/Learning_PyTorch/blob/main/CNN_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim

In [2]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
import pandas as pd
df=pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
torch.manual_seed(42)
X=df.iloc[:,1:]
y=df.iloc[:,0]

In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train=X_train/255.0
X_test=X_test/255.0

In [6]:
class FashionDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32).reshape(-1,1,28,28)
    self.labels=torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return(len(self.features))

  def __getitem__(self,index):
    return self.features[index],self.labels[index]


In [7]:
train_dataset=FashionDataset(X_train.values,y_train.values)
test_dataset=FashionDataset(X_test.values,y_test.values)

train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)


In [15]:
class MyCNN(nn.Module):
  def __init__(self,input_features):
    super().__init__()

    self.features=nn.Sequential(
        nn.Conv2d(input_features,32,kernel_size=3,padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(32,64,kernel_size=3,padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2,stride=2),
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7,128),
        nn.ReLU(),
        nn.Dropout(0.4),

        nn.Linear(128,64),
        nn.ReLU(),
        nn.Dropout(0.4),

        nn.Linear(64,10)
    )

  def forward(self,x):
    x=self.features(x)
    x=self.classifier(x)

    return x




In [13]:
learning_rate=0.1
epochs=50

In [14]:
model=MyCNN(1)
model.to(device)

criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=learning_rate,weight_decay=1e-4)

In [16]:
## training_loop

for epoch in range(epochs):
  total_epoch_loss=0

  for batch_features, batch_labels in train_loader:

    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)

    outputs=model(batch_features)

    loss=criterion(outputs,batch_labels)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    total_epoch_loss+=loss.item()

  avg_loss=total_epoch_loss/len(train_loader)
  print(f'Epoch:{epoch+1}, Loss:{avg_loss}')


Epoch:1, Loss:0.5966929829070965
Epoch:2, Loss:0.4132108386605978
Epoch:3, Loss:0.35404716326047975
Epoch:4, Loss:0.31606217089792094
Epoch:5, Loss:0.2870434202539424
Epoch:6, Loss:0.2666500890602668
Epoch:7, Loss:0.25286961246157685
Epoch:8, Loss:0.2367715447274968
Epoch:9, Loss:0.21879176389301816
Epoch:10, Loss:0.20955776783513527
Epoch:11, Loss:0.20314997140380245
Epoch:12, Loss:0.19521574291959404
Epoch:13, Loss:0.18363936237734743
Epoch:14, Loss:0.17496259248939652
Epoch:15, Loss:0.17193170918834705
Epoch:16, Loss:0.16605148414398233
Epoch:17, Loss:0.15590921044753245
Epoch:18, Loss:0.1548180549973622
Epoch:19, Loss:0.14719118266024936
Epoch:20, Loss:0.14684967152634634
Epoch:21, Loss:0.13959245129860937
Epoch:22, Loss:0.13575383645420272
Epoch:23, Loss:0.13507889166163903
Epoch:24, Loss:0.1329044932348188
Epoch:25, Loss:0.1306550921079567
Epoch:26, Loss:0.11807336669275537
Epoch:27, Loss:0.1211083192108199
Epoch:28, Loss:0.11588637183544537
Epoch:29, Loss:0.1154339486899941
Epoc

In [17]:
model.eval()

MyCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [18]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8943333333333333
